# LLM Eval: RAG over r/changemyview Conversations (Qwen 2.5)

**Goal:** load CSV from PRAW notebook, retrieve relevant branches, and prompt Qwen 2.5 to analyze *winning vs unsuccessful* rhetoric. Output structured JSON with confidence and evidence IDs.

**Sections:**
1. [Hardware & Model](#hw)  
2. [Load Data](#data)  
3. [Lite Retrieval](#retrieval)  
4. [Research Questions](#rqs)  
5. [Prompt & Generate](#prompt)  
6. [Parse JSON & Save](#save)  


<a id='hw'></a>

# Hardware & Model

Pick Qwen 2.5 size by available GPU memory (7B / 14B / 32B).

In [ ]:
# If needed:
# !pip install -q torch transformers pandas scikit-learn tiktoken

import torch, os, time
from transformers import AutoTokenizer, AutoModelForCausalLM

def pick_qwen_by_memory(vram_gb: float) -> str:
    if vram_gb >= 40:  # roomy
        return "Qwen/Qwen2.5-32B-Instruct"
    if vram_gb >= 18:
        return "Qwen/Qwen2.5-14B-Instruct"
    return "Qwen/Qwen2.5-7B-Instruct"

def detect_device():
    if torch.cuda.is_available():
        total = torch.cuda.get_device_properties(0).total_memory / (1024**3)
        return "cuda", total
    if hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
        return "mps", 8
    return "cpu", 0

DEVICE, VRAM_GB = detect_device()
MODEL_NAME = pick_qwen_by_memory(VRAM_GB)
DTYPE = torch.bfloat16 if DEVICE in {"cuda","mps"} else torch.float32

print(f"Device: {DEVICE}, VRAM≈{VRAM_GB:.1f} GB")
print(f"Model:  {MODEL_NAME}")

# Lazy load on first use to keep the notebook snappy
_tokenizer = _model = None
def load_model():
    global _tokenizer, _model
    if _model is None:
        _tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
        _model = AutoModelForCausalLM.from_pretrained(
            MODEL_NAME,
            torch_dtype=DTYPE,
            device_map="auto" if DEVICE!="cpu" else None,
        )
    return _tokenizer, _model

<a id='data'></a>

# Load Data

Uses the **same CSV schema** saved by the PRAW notebook.

In [ ]:
import pandas as pd
import os

CSV_PATH = "cmv_threads.csv"  # must match the PRAW notebook output
assert os.path.exists(CSV_PATH), f"CSV not found: {CSV_PATH} (run the PRAW notebook first)"
df = pd.read_csv(CSV_PATH)

# Minimal cleaning
df['text'] = df['body'].fillna("").astype(str)
print(df.head(2)[['post_id','comment_id','parent_id','branch_type','text']])

<a id='retrieval'></a>

# Lite Retrieval

Tiny TF‑IDF retrieval to keep things dependency‑light.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Make simple 'doc' units: each comment row is a doc.
docs = df[['post_id','comment_id','branch_type','text']].copy()
vec = TfidfVectorizer(min_df=2, max_df=0.95)
X = vec.fit_transform(docs['text'])

def retrieve(q: str, k: int = 6):
    qv = vec.transform([q])
    sims = cosine_similarity(qv, X).ravel()
    top = np.argsort(sims)[-k:][::-1]
    return docs.iloc[top].assign(score=sims[top])

<a id='rqs'></a>

# Research Questions

Start with a few templates or generate via the model.

In [ ]:
RQS = [
    "Which rhetorical strategies most often precede a granted delta?",
    "Do empathetic openers correlate with higher delta rates?",
    "Are hedging phrases more common in winning than unsuccessful branches?",
    "Does asking clarifying questions increase persuasion?",
    "How does citing personal experience affect outcomes?",
    "What tone differences (polite vs confrontational) predict success?",
]

<a id='prompt'></a>

# Prompt & Generate

Short helper to call Qwen and request **valid JSON**.

In [ ]:
import json, re

def format_context(rows):
    # Keep it tiny: id + short text
    out = []
    for _, r in rows.iterrows():
        snippet = r['text']
        if len(snippet) > 420:  # trim
            snippet = snippet[:420] + "…"
        out.append({
            "post_id": r['post_id'],
            "comment_id": r['comment_id'],
            "branch_type": r['branch_type'],
            "text": snippet
        })
    return out

SYSTEM = "You are a careful research assistant. Return ONLY JSON."

PROMPT_TMPL = (
    "You analyze persuasion in r/changemyview conversation snippets.\n"
    "Task: For the research question: {question}\n"
    "Use the provided snippets (some winning, some unsuccessful).\n"
    "Identify patterns, techniques, and cite evidence by comment_id.\n"
    "Return STRICT JSON with keys: "
    '{"question": "...", "claim": "...", "techniques": ["..."], '
    '"evidence_ids": ["comment_id", ...], "confidence": 0.0, "notes": "..."}\n\n'
    "Snippets JSON:\n{snippets}\n"
)

def generate_json_response(question: str, k: int = 6, max_new_tokens: int = 512):
    # get top-K docs per question, mix winning/unsuccessful if possible
    rows = retrieve(question, k=k)
    ctx = format_context(rows)
    prompt = PROMPT_TMPL.format(question=question, snippets=json.dumps(ctx, ensure_ascii=False))

    tok, mdl = load_model()
    inputs = tok(prompt, return_tensors="pt").to(mdl.device)
    with torch.no_grad():
        out = mdl.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=False)
    text = tok.decode(out[0], skip_special_tokens=True)

    # Try to extract JSON (robust to accidental pre/post text)
    m = re.search(r"\{[\s\S]*\}", text)
    if not m:
        raise ValueError("No JSON found in model output")
    payload = json.loads(m.group(0))
    payload["retrieved"] = ctx  # add context for traceability
    return payload

<a id='save'></a>

# Parse & Save

Run a few questions, write JSONL. Keep artifacts small.

In [ ]:
results = []
for q in RQS[:3]:  # demo; adjust as needed
    try:
        pj = generate_json_response(q, k=8)
        results.append(pj)
        print(f"✓ {q}")
    except Exception as e:
        print(f"⚠️ {q}: {e}")

out_path = "qwen_eval.jsonl"
with open(out_path, "w", encoding="utf-8") as f:
    for r in results:
        f.write(json.dumps(r, ensure_ascii=False) + "\n")

print(f"Saved {len(results)} records → {out_path}")